## A gentle 10-minute introduction to Ray AI Runitime (Ray AIR)

As part of Ray 2.0, Ray AI Runtime (AIR) is an open-source toolkit for building end-to-end ML applications. By leveraging Ray, its distributed compute capabilities, and its library ecosystem, Ray AIR brings scalability and programmability to ML platforms.

Ray AI Runtime focuses on two functional aspects:
 * It provides scalability by leveraging Ray’s distributed compute layer for ML workloads.
 * It is designed to interoperate with other systems for storage and metadata needs.

Ray AIR consists of five key Ray components:

 * Data processing ([Ray Data](https://docs.ray.io/en/latest/data/dataset.html))
 * Model Training ([Ray Train](https://docs.ray.io/en/latest/train/train.html))
 * Reinforcement Learning ([Ray RLlib](https://docs.ray.io/en/latest/rllib/index.html))
 * Hyperparameter Tuning ([Ray Tune](https://docs.ray.io/en/latest/tune/index.html))
 * Model Serving ([Ray Serve](https://docs.ray.io/en/latest/serve/index.html)).
 
 <img src = "images/ai_runtime.jpeg" width="60%" height="30%">
 
### Learning objectives:
  * Get introduced to Ray AIR as a unified toolkit to write an end-to-end ML application in a single Python script
  * Get exposed to Ray data for data ingestion
  * Learn about out-of-box Preprocessors
  * Load model from checkpoint and use for batch inference
  * Learn to deploy model and use for online inference

In [3]:
import logging, os, random, warnings
from pprint import pprint
import ray
import pandas as pd
from sklearn.model_selection import train_test_split

from ray.data.preprocessors import StandardScaler

In [4]:
warnings.filterwarnings("ignore")
os.environ["PYTHONWARNINGS"] = "ignore"

In [5]:
if ray.is_initialized:
    ray.shutdown()
context = ray.init(logging_level=logging.ERROR)
pprint(context)

RayContext(dashboard_url='127.0.0.1:8266', python_version='3.8.13', ray_version='3.0.0.dev0', ray_commit='{{RAY_COMMIT_SHA}}', address_info={'node_ip_address': '127.0.0.1', 'raylet_ip_address': '127.0.0.1', 'redis_address': None, 'object_store_address': '/tmp/ray/session_2022-07-20_09-59-17_235461_30511/sockets/plasma_store', 'raylet_socket_name': '/tmp/ray/session_2022-07-20_09-59-17_235461_30511/sockets/raylet', 'webui_url': '127.0.0.1:8266', 'session_dir': '/tmp/ray/session_2022-07-20_09-59-17_235461_30511', 'metrics_export_port': 64493, 'gcs_address': '127.0.0.1:62846', 'address': '127.0.0.1:62846', 'dashboard_agent_listen_port': 52365, 'node_id': '77b154b5e9078df027f0fcfb0cb5db7cd3b472867f54b9d847eaf155'})


In [6]:
print(f"Dashboard url: http://{context.address_info['webui_url']}")

Dashboard url: http://127.0.0.1:8266


### Create Ray data from an S3 CSV datasource

In [9]:
import ray
import pandas as pd
from ray.air import train_test_split

# Split data into train and validation.
dataset = ray.data.read_csv("s3://anonymous@air-example-data/breast_cancer.csv")
train_dataset, valid_dataset = train_test_split(dataset, test_size=0.3)
test_dataset = valid_dataset.drop_columns(["target"])

Map_Batches: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 76.46it/s]


### Create Preprocessors
This preprocessor is automatically used in the training function to `fit` and `tranform` your datasets for training and validation.

In [11]:
# Create a preprocessor to scale some columns
from ray.data.preprocessors import StandardScaler

columns_to_scale = ["mean radius", "mean texture"]
preprocessor = StandardScaler(columns=columns_to_scale)

### Create Trainers
Use the Ray AIR trainer `XGBoostTrainer`. Simple steps:
 1. define the parallelism for Ray compute
 2. define the XGBoost parameters for training
 3. supply the preprocessor for fitting and transforming dataset during training and validation
 4. provide the datasets training and validation
 5. invoke `trainer.fit()` 
 
 Simple API that does a lot behind the scenes!

In [12]:
from ray.train.xgboost import XGBoostTrainer

trainer = XGBoostTrainer(
    scaling_config={
        # Number of workers to use for data parallelism.
        "num_workers": 2,
        # Whether to use GPU acceleration.
        "use_gpu": False,
    },
    label_column="target",
    num_boost_round=20,
    params={
        # XGBoost specific params
        "objective": "binary:logistic",
        "eval_metric": ["logloss", "error"],
    },
    datasets={"train": train_dataset, "valid": valid_dataset},
    preprocessor=preprocessor,
)
result = trainer.fit()
print(result.metrics)

Trial name,status,loc,iter,total time (s),train-logloss,train-error,valid-logloss
XGBoostTrainer_89eea_00000,TERMINATED,127.0.0.1:32882,20,4.51621,0.0184957,0,0.0893879


(_map_block_nosplit pid=32033) E0720 10:08:13.821844000 6132838400 chttp2_transport.cc:1111]          Received a GOAWAY with error code ENHANCE_YOUR_CALM and debug data equal to "too_many_pings"
(XGBoostTrainer pid=32882) 2022-07-20 10:08:15,980	WARNING base_trainer.py:162 -- When passing `datasets` to a Trainer, it is recommended to reserve at least 20% of node CPUs for Dataset execution by setting `_max_cpu_fraction_per_node = 0.8` in the Trainer `scaling_config`. Not doing so can lead to resource contention or hangs. See https://docs.ray.io/en/master/data/key-concepts.html#example-datasets-in-tune for more info.
(XGBoostTrainer pid=32882) 2022-07-20 10:08:17,540	INFO main.py:980 -- [RayXGBoost] Created 2 new actors (2 total actors). Waiting until actors are ready for training.
(_RemoteRayXGBoostActor pid=32894) 2022-07-20 10:08:18,521	WARNING __init__.py:192 -- DeprecationWarning: `ray.worker.get_resource_ids` is a private attribute and access will be removed in a future Ray version

Result for XGBoostTrainer_89eea_00000:
  date: 2022-07-20_10-08-20
  done: false
  experiment_id: 8a44aa699d134bbca129d68b4e4f7ae6
  hostname: Juless-MacBook-Pro-16
  iterations_since_restore: 1
  node_ip: 127.0.0.1
  pid: 32882
  should_checkpoint: true
  time_since_restore: 4.391774892807007
  time_this_iter_s: 4.391774892807007
  time_total_s: 4.391774892807007
  timestamp: 1658336900
  timesteps_since_restore: 0
  train-error: 0.02261306532663317
  train-logloss: 0.464117960489575
  training_iteration: 1
  trial_id: 89eea_00000
  valid-error: 0.11695906432748537
  valid-logloss: 0.5025240946234318
  warmup_time: 0.0025348663330078125
  


(XGBoostTrainer pid=32882) 2022-07-20 10:08:20,500	INFO main.py:1516 -- [RayXGBoost] Finished XGBoost training on training data with total N=398 in 2.97 seconds (1.79 pure XGBoost training time).


Result for XGBoostTrainer_89eea_00000:
  date: 2022-07-20_10-08-20
  done: true
  experiment_id: 8a44aa699d134bbca129d68b4e4f7ae6
  experiment_tag: '0'
  hostname: Juless-MacBook-Pro-16
  iterations_since_restore: 20
  node_ip: 127.0.0.1
  pid: 32882
  should_checkpoint: true
  time_since_restore: 4.51620888710022
  time_this_iter_s: 0.0064239501953125
  time_total_s: 4.51620888710022
  timestamp: 1658336900
  timesteps_since_restore: 0
  train-error: 0.0
  train-logloss: 0.01849572773292735
  training_iteration: 20
  trial_id: 89eea_00000
  valid-error: 0.04093567251461988
  valid-logloss: 0.08938791319913073
  warmup_time: 0.0025348663330078125
  
{'train-logloss': 0.01849572773292735, 'train-error': 0.0, 'valid-logloss': 0.08938791319913073, 'valid-error': 0.04093567251461988, 'time_this_iter_s': 0.0064239501953125, 'should_checkpoint': True, 'done': True, 'timesteps_total': None, 'episodes_total': None, 'training_iteration': 20, 'trial_id': '89eea_00000', 'experiment_id': '8a44aa69

### Create Tuner for hyperparameter search

What if you want to do some hyperparameter optimization during training and use the best config for the model? 
Well, you can then use Tuner and supply your training function as part of the argument, along with other Tuner
configuration. 

Again, simple steps:
 1. define your hyperparameter space
 2. define `TuneConfig` for number of trials and parallelism 
 3. invoke `tuner.fit()`

In [14]:
from ray import tune

param_space = {"params": {"max_depth": tune.randint(1, 9)}}
metric = "train-logloss"

In [15]:
from ray.tune.tuner import Tuner, TuneConfig
from ray.air.config import RunConfig

tuner = Tuner(
    trainer,
    param_space=param_space,
    tune_config=TuneConfig(num_samples=5, metric=metric, mode="min"),
)
# Execute tuning.
result_grid = tuner.fit()

# Fetch the best result.That is its best hyperparameter config
best_result = result_grid.get_best_result()
print("Best Result:", best_result)

Trial name,status,loc,params/max_depth,iter,total time (s),train-logloss,train-error,valid-logloss
XGBoostTrainer_4ec83_00000,TERMINATED,127.0.0.1:33502,5,20,4.61511,0.0184163,0,0.105782
XGBoostTrainer_4ec83_00001,TERMINATED,127.0.0.1:33506,3,20,3.43237,0.0215151,0,0.0765915
XGBoostTrainer_4ec83_00002,TERMINATED,127.0.0.1:33507,5,20,3.7597,0.0184163,0,0.105782
XGBoostTrainer_4ec83_00003,TERMINATED,127.0.0.1:33566,3,20,4.62312,0.0215151,0,0.0765915
XGBoostTrainer_4ec83_00004,TERMINATED,127.0.0.1:33570,5,20,4.04092,0.0184163,0,0.105782


(_block_to_df pid=32889) E0720 10:13:44.790356000 6199128064 chttp2_transport.cc:1111]          Received a GOAWAY with error code ENHANCE_YOUR_CALM and debug data equal to "too_many_pings"
(XGBoostTrainer pid=33502) 2022-07-20 10:13:44,785	WARNING base_trainer.py:162 -- When passing `datasets` to a Trainer, it is recommended to reserve at least 20% of node CPUs for Dataset execution by setting `_max_cpu_fraction_per_node = 0.8` in the Trainer `scaling_config`. Not doing so can lead to resource contention or hangs. See https://docs.ray.io/en/master/data/key-concepts.html#example-datasets-in-tune for more info.
(XGBoostTrainer pid=33502) 2022-07-20 10:13:45,902	INFO main.py:980 -- [RayXGBoost] Created 2 new actors (2 total actors). Waiting until actors are ready for training.
(XGBoostTrainer pid=33506) 2022-07-20 10:13:45,918	WARNING base_trainer.py:162 -- When passing `datasets` to a Trainer, it is recommended to reserve at least 20% of node CPUs for Dataset execution by setting `_max_c

Result for XGBoostTrainer_4ec83_00001:
  date: 2022-07-20_10-13-49
  done: false
  experiment_id: ab445a1094b043e795d8d51ba583c4e0
  hostname: Juless-MacBook-Pro-16
  iterations_since_restore: 1
  node_ip: 127.0.0.1
  pid: 33506
  should_checkpoint: true
  time_since_restore: 3.255877733230591
  time_this_iter_s: 3.255877733230591
  time_total_s: 3.255877733230591
  timestamp: 1658337229
  timesteps_since_restore: 0
  train-error: 0.03517587939698492
  train-logloss: 0.47431553248784053
  training_iteration: 1
  trial_id: 4ec83_00001
  valid-error: 0.09941520467836257
  valid-logloss: 0.5004687657830311
  warmup_time: 0.002753019332885742
  
Result for XGBoostTrainer_4ec83_00000:
  date: 2022-07-20_10-13-49
  done: false
  experiment_id: 68acc7455e6240d1a27e911f52c4a1f3
  hostname: Juless-MacBook-Pro-16
  iterations_since_restore: 1
  node_ip: 127.0.0.1
  pid: 33502
  should_checkpoint: true
  time_since_restore: 4.445180892944336
  time_this_iter_s: 4.445180892944336
  time_total_s: 4

(XGBoostTrainer pid=33506) 2022-07-20 10:13:49,358	INFO main.py:1516 -- [RayXGBoost] Finished XGBoost training on training data with total N=398 in 3.39 seconds (1.97 pure XGBoost training time).
(XGBoostTrainer pid=33502) 2022-07-20 10:13:49,406	INFO main.py:1516 -- [RayXGBoost] Finished XGBoost training on training data with total N=398 in 3.52 seconds (2.05 pure XGBoost training time).


Result for XGBoostTrainer_4ec83_00000:
  date: 2022-07-20_10-13-49
  done: true
  experiment_id: 68acc7455e6240d1a27e911f52c4a1f3
  experiment_tag: 0_max_depth=5
  hostname: Juless-MacBook-Pro-16
  iterations_since_restore: 20
  node_ip: 127.0.0.1
  pid: 33502
  should_checkpoint: true
  time_since_restore: 4.615113973617554
  time_this_iter_s: 0.008207082748413086
  time_total_s: 4.615113973617554
  timestamp: 1658337229
  timesteps_since_restore: 0
  train-error: 0.0
  train-logloss: 0.01841634292981527
  training_iteration: 20
  trial_id: 4ec83_00000
  valid-error: 0.05263157894736842
  valid-logloss: 0.10578184703239703
  warmup_time: 0.002573728561401367
  
Result for XGBoostTrainer_4ec83_00002:
  date: 2022-07-20_10-13-49
  done: false
  experiment_id: dfd4553c9c414a6ab9b6fa38ba5f2ed9
  hostname: Juless-MacBook-Pro-16
  iterations_since_restore: 1
  node_ip: 127.0.0.1
  pid: 33507
  should_checkpoint: true
  time_since_restore: 3.637022018432617
  time_this_iter_s: 3.637022018432

...

Result for XGBoostTrainer_4ec83_00004:
  date: 2022-07-20_10-13-55
  done: false
  experiment_id: 817793337ab145c1a2864b6c606638ff
  hostname: Juless-MacBook-Pro-16
  iterations_since_restore: 1
  node_ip: 127.0.0.1
  pid: 33570
  should_checkpoint: true
  time_since_restore: 3.8910109996795654
  time_this_iter_s: 3.8910109996795654
  time_total_s: 3.8910109996795654
  timestamp: 1658337235
  timesteps_since_restore: 0
  train-error: 0.02261306532663317
  train-logloss: 0.465611254524945
  training_iteration: 1
  trial_id: 4ec83_00004
  valid-error: 0.0935672514619883
  valid-logloss: 0.5058815336366843
  warmup_time: 0.002708911895751953
  


(XGBoostTrainer pid=33570) 2022-07-20 10:13:55,578	INFO main.py:1516 -- [RayXGBoost] Finished XGBoost training on training data with total N=398 in 3.18 seconds (1.96 pure XGBoost training time).


Result for XGBoostTrainer_4ec83_00003:
  date: 2022-07-20_10-13-55
  done: false
  experiment_id: ce8d9fe83d284ad2aec1013b7d8a2d6c
  hostname: Juless-MacBook-Pro-16
  iterations_since_restore: 1
  node_ip: 127.0.0.1
  pid: 33566
  should_checkpoint: true
  time_since_restore: 4.509383916854858
  time_this_iter_s: 4.509383916854858
  time_total_s: 4.509383916854858
  timestamp: 1658337235
  timesteps_since_restore: 0
  train-error: 0.03517587939698492
  train-logloss: 0.47431553248784053
  training_iteration: 1
  trial_id: 4ec83_00003
  valid-error: 0.09941520467836257
  valid-logloss: 0.5004687657830311
  warmup_time: 0.0027549266815185547
  


(XGBoostTrainer pid=33566) 2022-07-20 10:13:55,830	INFO main.py:1516 -- [RayXGBoost] Finished XGBoost training on training data with total N=398 in 2.99 seconds (1.79 pure XGBoost training time).


Result for XGBoostTrainer_4ec83_00004:
  date: 2022-07-20_10-13-55
  done: true
  experiment_id: 817793337ab145c1a2864b6c606638ff
  experiment_tag: 4_max_depth=5
  hostname: Juless-MacBook-Pro-16
  iterations_since_restore: 20
  node_ip: 127.0.0.1
  pid: 33570
  should_checkpoint: true
  time_since_restore: 4.04092001914978
  time_this_iter_s: 0.005606174468994141
  time_total_s: 4.04092001914978
  timestamp: 1658337235
  timesteps_since_restore: 0
  train-error: 0.0
  train-logloss: 0.01841634292981527
  training_iteration: 20
  trial_id: 4ec83_00004
  valid-error: 0.05263157894736842
  valid-logloss: 0.10578184703239703
  warmup_time: 0.002708911895751953
  
Result for XGBoostTrainer_4ec83_00003:
  date: 2022-07-20_10-13-55
  done: true
  experiment_id: ce8d9fe83d284ad2aec1013b7d8a2d6c
  experiment_tag: 3_max_depth=3
  hostname: Juless-MacBook-Pro-16
  iterations_since_restore: 20
  node_ip: 127.0.0.1
  pid: 33566
  should_checkpoint: true
  time_since_restore: 4.623118162155151
  ti

### Create a `BatchPreditor` for batch prediction
Once you have trained and tuned your model, create a batch predictor from best model using the `best_result.checkpoint` and do batch inference. 

In [18]:
from ray.train.batch_predictor import BatchPredictor
from ray.train.xgboost import XGBoostPredictor

batch_predictor = BatchPredictor.from_checkpoint(best_result.checkpoint, XGBoostPredictor)

predicted_probabilities = batch_predictor.predict(test_dataset)
print("PREDICTED PROBABILITIES")
predicted_probabilities.show()

Map Progress (1 actors 1 pending): 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  1.37it/s]

PREDICTED PROBABILITIES
{'predictions': 0.9960426092147827}
{'predictions': 0.9957077503204346}
{'predictions': 0.0034389763604849577}
{'predictions': 0.9962536096572876}
{'predictions': 0.9968380928039551}
{'predictions': 0.9957551956176758}
{'predictions': 0.9920042157173157}
{'predictions': 0.994161069393158}
{'predictions': 0.2891101539134979}
{'predictions': 0.974367082118988}
{'predictions': 0.0034389763604849577}
{'predictions': 0.9959942102432251}
{'predictions': 0.9474029541015625}
{'predictions': 0.9923243522644043}
{'predictions': 0.9941523671150208}
{'predictions': 0.1239369809627533}
{'predictions': 0.5043733716011047}
{'predictions': 0.9935414791107178}
{'predictions': 0.9832899570465088}
{'predictions': 0.0034389763604849577}


### Create `PredictorDeployment` for Online Inference
Deploy the best model as an inference service by using Ray Serve and the `PredictorDeployment` class.

In [19]:
from ray import serve
from fastapi import Request
from ray.serve import PredictorDeployment
from ray.serve.http_adapters import json_request


async def adapter(request: Request):
    content = await request.json()
    print(content)
    return pd.DataFrame.from_dict(content)


serve.start(detached=True)
deployment = PredictorDeployment.options(name="XGBoostService")

deployment.deploy(
    XGBoostPredictor, best_result.checkpoint, batching_params=False, http_adapter=adapter
)

print(deployment.url)

(ServeController pid=34551) INFO 2022-07-20 10:24:02,391 controller 34551 checkpoint_path.py:17 - Using RayInternalKVStore for controller checkpoint and recovery.
(ServeController pid=34551) INFO 2022-07-20 10:24:02,419 controller 34551 http_state.py:123 - Starting HTTP proxy with name 'SERVE_CONTROLLER_ACTOR:SERVE_PROXY_ACTOR-77b154b5e9078df027f0fcfb0cb5db7cd3b472867f54b9d847eaf155' on node '77b154b5e9078df027f0fcfb0cb5db7cd3b472867f54b9d847eaf155' listening on '127.0.0.1:8000'
(HTTPProxyActor pid=34553) INFO:     Started server process [34553]
(ServeController pid=34551) INFO 2022-07-20 10:24:03,041 controller 34551 deployment_state.py:1280 - Adding 1 replicas to deployment 'XGBoostService'.


http://127.0.0.1:8000/XGBoostService


After deploying the service, you can send requests to it.

In [20]:
import requests

sample_input = test_dataset.take(1)
sample_input = dict(sample_input[0])

output = requests.post(deployment.url, json=[sample_input]).json()
print(output)

[{'predictions': 0.9960426092147827}]
(ServeReplica:XGBoostService pid=34555) [{'mean radius': 11.06, 'mean texture': 14.83, 'mean perimeter': 70.31, 'mean area': 378.2, 'mean smoothness': 0.07741, 'mean compactness': 0.04768, 'mean concavity': 0.02712, 'mean concave points': 0.007246, 'mean symmetry': 0.1535, 'mean fractal dimension': 0.06214, 'radius error': 0.1855, 'texture error': 0.6881, 'perimeter error': 1.263, 'area error': 12.98, 'smoothness error': 0.004259, 'compactness error': 0.01469, 'concavity error': 0.0194, 'concave points error': 0.004168, 'symmetry error': 0.01191, 'fractal dimension error': 0.003537, 'worst radius': 12.68, 'worst texture': 20.35, 'worst perimeter': 80.79, 'worst area': 496.7, 'worst smoothness': 0.112, 'worst compactness': 0.1879, 'worst concavity': 0.2079, 'worst concave points': 0.05556, 'worst symmetry': 0.259, 'worst fractal dimension': 0.09158}]


(HTTPProxyActor pid=34553) INFO 2022-07-20 10:24:15,872 http_proxy 127.0.0.1 http_proxy.py:315 - POST /XGBoostService 307 5.7ms
(HTTPProxyActor pid=34553) INFO 2022-07-20 10:24:15,880 http_proxy 127.0.0.1 http_proxy.py:315 - POST /XGBoostService 200 6.3ms
(ServeReplica:XGBoostService pid=34555) INFO 2022-07-20 10:24:15,871 XGBoostService XGBoostService#LVqWvb replica.py:478 - HANDLE __call__ OK 0.3ms
(ServeReplica:XGBoostService pid=34555) INFO 2022-07-20 10:24:15,879 XGBoostService XGBoostService#LVqWvb replica.py:478 - HANDLE __call__ OK 4.4ms


In [21]:
ray.shutdown()